In [4]:
import h5py

with h5py.File(model_path, 'r') as f:
    print("Saved layer names:")
    for layer in f['model_weights']:
        print(layer)


Saved layer names:
bidirectional
breathing_attention
conv1d
dense
dense_1
dense_2
dropout
dropout_1
dropout_2
input_1
layer_normalization
layer_normalization_1
max_pooling1d
multi_head_attention
tf.__operators__.add
tf.__operators__.add_1
top_level_model_weights


In [14]:
print(h5py.__version__)

3.12.1


In [15]:
print(h5py.__file__)

c:\Users\ADWIN ADI\AppData\Local\Programs\Python\Python39\lib\site-packages\h5py\__init__.py


In [43]:
from tensorflow.keras.models import load_model

# Path to your model
model_path = r"C:\Users\ADWIN ADI\OneDrive\Desktop\Jupyter notebook projects\RESP_MODELS\RESP_MODEL_PROJECT_COUSTOMIZED\breathing_model.h5"

# Load with custom object
model = load_model(model_path, custom_objects={'BreathingAttention': BreathingAttention})

print(" Model loaded successfully!")
model.summary()


 Model loaded successfully!
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 450, 6)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 450, 64)      1984        ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 225, 64)      0           ['conv1d[0][0]']                 
                                                                                                  
 bidirectional (Bidirectional)  (None, 225, 128)     66048       ['max_pooling1d[0][0]']          
                                                                  

In [74]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class BreathingAttention(Layer):
    def __init__(self, **kwargs):
        super(BreathingAttention, self).__init__(**kwargs)

    def call(self, inputs):
        # Apply softmax across time steps (axis=1)
        attention_weights = tf.nn.softmax(inputs, axis=1)
        output = inputs * attention_weights
        return output

    def get_config(self):
        config = super().get_config()
        return config


In [76]:
from tensorflow.keras.models import load_model

# Register the custom layer
model = load_model(r"C:\Users\ADWIN ADI\OneDrive\Desktop\Jupyter notebook projects\RESP_MODELS\RESP_MODEL_PROJECT_COUSTOMIZED\breathing_model.h5",custom_objects={'BreathingAttention': BreathingAttention})


ValueError: Layer count mismatch when loading weights from file. Model expected 8 layers, found 9 saved layers.

In [78]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the existing model with the custom layer
model = load_model(r"C:\Users\ADWIN ADI\OneDrive\Desktop\Jupyter notebook projects\RESP_MODELS\RESP_MODEL_PROJECT_COUSTOMIZED\breathing_model.h5", custom_objects={'BreathingAttention': BreathingAttention})

# Optional: print summary to check model architecture
model.summary()

# Compile again if needed
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-6)

# Train on new data
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=50,
          batch_size=32,
          callbacks=[early_stop, reduce_lr])

# Save the updated model
model.save("breathing_updated.h5")


ValueError: Layer count mismatch when loading weights from file. Model expected 8 layers, found 9 saved layers.

In [80]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import tensorflow as tf

# Define the custom layer exactly how it was defined when saved
class BreathingAttention(Layer):
    def __init__(self, **kwargs):
        super(BreathingAttention, self).__init__(**kwargs)

    def call(self, inputs):
        attention = tf.nn.softmax(inputs, axis=1)
        return inputs * attention

    def get_config(self):
        return super().get_config()

# Now load the model
model = load_model(
    r"C:\Users\ADWIN ADI\OneDrive\Desktop\Jupyter notebook projects\RESP_MODELS\RESP_MODEL_PROJECT_COUSTOMIZED\breathing_model.h5",
    custom_objects={'BreathingAttention': BreathingAttention}
)


ValueError: Layer count mismatch when loading weights from file. Model expected 8 layers, found 9 saved layers.

In [81]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Layer, Multiply
from tensorflow.keras.utils import plot_model

# Custom Attention Layer
class BreathingAttention(Layer):
    def __init__(self, **kwargs):
        super(BreathingAttention, self).__init__(**kwargs)

    def call(self, inputs):
        attention = tf.nn.softmax(inputs, axis=1)
        return inputs * attention

    def get_config(self):
        return super().get_config()

# Build Model with 9 Layers
def build_breathing_model(input_shape=(100, 1)):
    inputs = Input(shape=input_shape, name="Input_Layer")                                # Layer 1

    x = LSTM(64, return_sequences=True, name="LSTM_1")(inputs)                           # Layer 2
    x = Dropout(0.3, name="Dropout_1")(x)                                                 # Layer 3

    x = LSTM(32, return_sequences=True, name="LSTM_2")(x)                                 # Layer 4
    x = Dropout(0.3, name="Dropout_2")(x)                                                 # Layer 5

    x = BreathingAttention(name="BreathingAttention")(x)                                  # Layer 6

    x = tf.keras.layers.GlobalAveragePooling1D(name="GlobalAvgPool")(x)                  # Layer 7
    x = Dense(32, activation='relu', name="Dense_1")(x)                                   # Layer 8
    outputs = Dense(3, activation='softmax', name="Output")(x)                            # Layer 9

    model = Model(inputs, outputs, name="Breathing_Model")
    return model

# Instantiate the model
model = build_breathing_model()

# Summary to verify
model.summary()

# Optional: Save if you want a fresh .h5 model
# model.save("breathing_model_fresh_9_layers.h5")


Model: "Breathing_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (InputLayer)    [(None, 100, 1)]          0         
                                                                 
 LSTM_1 (LSTM)               (None, 100, 64)           16896     
                                                                 
 Dropout_1 (Dropout)         (None, 100, 64)           0         
                                                                 
 LSTM_2 (LSTM)               (None, 100, 32)           12416     
                                                                 
 Dropout_2 (Dropout)         (None, 100, 32)           0         
                                                                 
 BreathingAttention (Breathi  (None, 100, 32)          0         
 ngAttention)                                                    
                                                   